# 2D Magnetodynamics (Time-Harmonic) - Distribution Transformer

In [1]:
include(joinpath(@__DIR__, "config.jl"))
paths = get_project_paths("examples")

# Ensure the module is reloaded if changed
if isdefined(Main, :MagnetostaticsFEM)
    println("Reloading MagnetostaticsFEM...")
    # A simple way to force reload in interactive sessions
    try; delete!(LOAD_PATH, joinpath(paths["SRC_DIR"], "src")); catch; end
    try; delete!(Base.loaded_modules, Base.PkgId(Base.UUID("f8a2b3c4-d5e6-f7a8-b9c0-d1e2f3a4b5c6"), "MagnetostaticsFEM")); catch; end
end
include(joinpath(paths["SRC_DIR"], "MagnetostaticsFEM.jl"))

using LinearAlgebra
using Plots
using LaTeXStrings
using Gridap
using GridapGmsh: GmshDiscreteModel
using .MagnetostaticsFEM
using Printf # For animation title formatting

┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.WeakFormProblem :: Union{}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetostatics_1d_weak_form :: Tuple{Gridap.Geometry.Triangulation, Gridap.CellData.Measure, AbstractArray, Any, Any}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetodynamics_1d_harmonic_weak_form :: Tuple{Gridap.Geometry.Triangulation, Gridap.CellData.Measure, AbstractArray, Any, Any, Any, Float64}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetodynamics_1d_harmonic_coupled_weak_form :: Tuple{Gridap.Geometry.Triangulation, Gridap.CellData.Measure, AbstractArray, Any, Any, Any, Float64}` in module `Main.MagnetostaticsFEM`
└ @ Base.Docs docs/Docs.jl:243
┌ Warning: Replacing docs for `Main.MagnetostaticsFEM.magnetodynamics_2d_harmon

## Define Parameters and Paths

In [ ]:
# Model Parameters
μ0 = 4e-7 * pi  # Vacuum permeability [H/m]
μr_core = 1000.0 # Relative permeability of the core (linear case)
σ_core = 0.1    # Conductivity of the core [S/m] (Laminated)
freq = 50.0     # Frequency [Hz]
ω = 2 * pi * freq # Angular frequency [rad/s]

# Source Parameters (Secondary Current Driven, Primary Open)
Ip = 0.0       # Primary peak phase current [A]
Is = 777.62    # Secondary peak phase current [A]
Np = 266       # Number of primary turns
Ns = 6         # Number of secondary turns

# Winding Areas (Approximate - needed for current density)
# These values seem missing in the original notebook, using placeholders.
# Need to calculate these from the geometry definition if possible, or use values from source.
Awhv_approx = 6000e-6 # Approximate area HV winding region [m^2]
Awlv_approx = 3000e-6 # Approximate area LV winding region [m^2]

Jp = Np * Ip / Awhv_approx # Peak current density primary [A/m^2]
Js = Ns * Is / Awlv_approx # Peak current density secondary [A/m^2]

# FEM Parameters
order = 1 # Linear elements match original notebook
field_type = ComplexF64 # Use ComplexF64 marker for setup_fe_spaces
dirichlet_tag_name = "boundary" # Name of the physical group for Dirichlet BC
dirichlet_value = 0.0 + 0.0im # Dirichlet BC for Az = u + iv

# Paths
mesh_file = joinpath(paths["GEO_DIR"], "transformer_stedin.msh")
output_file_base = joinpath(paths["OUTPUT_DIR"], "transformer_linear_dynamics")

println("Mesh file: ", mesh_file)
println("Output directory: ", paths["OUTPUT_DIR"])

Mesh file: /Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/ta_example_2d_distribution_transformer/geo/transformer_stedin.msh
Output directory: /Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/examples/output


## Setup FEM Problem (Linear Magnetodynamics)

In [3]:
# Load mesh and tags
model, labels, tags = load_mesh_and_tags(mesh_file)

# Get material tags dictionary
material_tags = get_material_tags(labels)
println("Material Tags: ", material_tags)

# Set up triangulation and measures
Ω = Triangulation(model)
dΩ = Measure(Ω, 2*order)

# Define material property functions
reluctivity_func = define_reluctivity(material_tags, μ0, μr_core; core_tag_name="Core")
conductivity_func = define_conductivity(material_tags, σ_core; core_tag_name="Core")

# Define source current density function (complex phasors)
# Jz = Jp*phase_p*(winding_p_neg - winding_p_pos) + Js*phase_s*(winding_s_pos - winding_s_neg)
function source_current_density_2d(tags_dict, Jp, Js)
    tag_hv1_l = tags_dict["HV winding phase 1 left"]
    tag_hv1_r = tags_dict["HV winding phase 1 right"]
    tag_hv2_l = tags_dict["HV winding phase 2 left"]
    tag_hv2_r = tags_dict["HV winding phase 2 right"]
    tag_hv3_l = tags_dict["HV winding phase 3 left"]
    tag_hv3_r = tags_dict["HV winding phase 3 right"]
    tag_lv1_l = tags_dict["LV winding phase 1 left"]
    tag_lv1_r = tags_dict["LV winding phase 1 right"]
    tag_lv2_l = tags_dict["LV winding phase 2 left"]
    tag_lv2_r = tags_dict["LV winding phase 2 right"]
    tag_lv3_l = tags_dict["LV winding phase 3 left"]
    tag_lv3_r = tags_dict["LV winding phase 3 right"]
    
    phase1 = exp(1im * 2pi/3)
    phase2 = 1.0 + 0.0im
    phase3 = exp(-1im * 2pi/3)
    
    function Jz(tag)
        current = 0.0 + 0.0im
        # Primary (HV)
        if tag == tag_hv1_l; current -= Jp * phase1; end
        if tag == tag_hv1_r; current += Jp * phase1; end
        if tag == tag_hv2_l; current -= Jp * phase2; end
        if tag == tag_hv2_r; current += Jp * phase2; end
        if tag == tag_hv3_l; current -= Jp * phase3; end
        if tag == tag_hv3_r; current += Jp * phase3; end
        # Secondary (LV)
        if tag == tag_lv1_l; current += Js * phase1; end
        if tag == tag_lv1_r; current -= Js * phase1; end
        if tag == tag_lv2_l; current += Js * phase2; end
        if tag == tag_lv2_r; current -= Js * phase2; end
        if tag == tag_lv3_l; current += Js * phase3; end
        if tag == tag_lv3_r; current -= Js * phase3; end
        return current
    end
    return Jz
end

# Note: The original notebook assumes J0 is real and uses the coupled formulation.
# Here, we define J0 potentially complex, but the coupled formulation expects real J0.
# We need to adjust the formulation or the source definition.
# Let's assume J0 is real for now, matching the 1D example's coupled form.
# We'll use the magnitude of Js for phase 2 as the reference J0.
J0_real_ref = Js # Use magnitude of secondary current density as reference
source_current_func_real = define_current_density(material_tags, J0_real_ref; 
                                            coil_tags_pos=["LV winding phase 2 left"], 
                                            coil_tags_neg=["LV winding phase 2 right"])
# TODO: This is a simplification. A full complex source requires a complex weak form or modification of the coupled form.

# Setup FE spaces (multi-field: Real, Imag parts)
dirichlet_tag_id = get_tag_from_name(labels, dirichlet_tag_name)
U, V = setup_fe_spaces(model, order, field_type, [dirichlet_tag_id], dirichlet_value)

# Define the weak form problem for the coupled system
problem = magnetodynamics_2d_harmonic_coupled_weak_form(Ω, dΩ, tags, reluctivity_func, conductivity_func, source_current_func_real, ω)

Info    : Reading '/Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/ta_example_2d_distribution_transformer/geo/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10393 nodes
Info    : 20784 elements
Info    : Done reading '/Users/ezracerpac/PycharmProjects/FutureDistributionSystemsAM/ta_example_2d_distribution_transformer/geo/transformer_stedin.msh'


LoadError: Unknown tag name Air

## Solve FEM Problem

In [ ]:
# Solve the real coupled linear FE system
uv = solve_fem_problem(problem, U, V) # uv is a MultiFieldFEFunction

# Extract real and imaginary parts
u = uv[1] # Real part of Az
v = uv[2] # Imag part of Az

## Post-processing

In [ ]:
# Compute B-field (Real and Imag parts)
B_re, B_im = calculate_b_field(uv)

# Compute Eddy Currents (Real and Imag parts)
J_eddy_re, J_eddy_im = calculate_eddy_current(uv, conductivity_func, ω, Ω, tags)

# Define helper functions for magnitude squared
mag_sq_scalar(re, im) = re*re + im*im
mag_sq_vector(re, im) = inner(re, re) + inner(im, im)

# Calculate Magnitudes for saving/plotting using composition
Az_mag = sqrt ∘ (mag_sq_scalar ∘ (u, v))
B_mag = sqrt ∘ (mag_sq_vector ∘ (B_re, B_im))
Jeddy_mag = sqrt ∘ (mag_sq_scalar ∘ (J_eddy_re, J_eddy_im))

# Calculate total current density magnitude (approximation)
# J_total_re = source_current_func_real(tags) + J_eddy_re # Need CellField source
# J_total_im = J_eddy_im
# J_total_mag = sqrt ∘ (mag_sq_scalar ∘ (J_total_re, J_total_im))
# TODO: Need a better way to handle source current in post-processing

# Save results to VTK format
save_results_vtk(Ω, output_file_base, 
    Dict(
        "Az_re" => u, "Az_im" => v, "Az_mag" => Az_mag,
        "B_re" => B_re, "B_im" => B_im, "B_mag" => B_mag,
        "Jeddy_re" => J_eddy_re, "Jeddy_im" => J_eddy_im, "Jeddy_mag" => Jeddy_mag
        # "Jtotal_mag" => J_total_mag
    ))

## Next Steps

1.  **Refine Source Term:** Implement the full complex source current density, potentially requiring a modification to the weak form or using a direct complex formulation if `Gridap` supports it easily.
2.  **Non-Linearity:** Extend `MagnetostaticsFEM.jl` to handle non-linear reluctivity (BH curve) using an iterative solver.
3.  **Voltage-Driven Coils:** Extend `MagnetostaticsFEM.jl` to handle coupled circuit equations using `MultiFieldFESpace`.